# Definition of the model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt
from constants import *
from GTZAN import GTZAN
from visualization import *
import time
import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cpu


In [9]:
class BeatTracker(nn.Module):
    
    def __init__(self, hidden_size=128, num_layers=2):
        super(BeatTracker, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
                        nb, 
                        hidden_size, 
                        num_layers, 
                        bidirectional=True, 
                        dropout=0.5,
                        batch_first=True)
        self.hid_to_beat = nn.Linear(2 * hidden_size, 2)
        self.hidden = None
        
        self.loss_function = nn.NLLLoss()
        
        self.lr = 0.001
        self.optimizer = optim.Adam(self.parameters(), lr=self.lr)

    def forward(self, spec):
        x = self.lstm(spec)[0]
        x = self.hid_to_beat(x)
        x = F.log_softmax(x, dim=-1)
        return x
    
    def set_lr(self, lr):
        self.lr = lr
        for p in self.optimizer.param_groups:
            p['lr'] = lr
    
    def loss(self, spec, onsets, isbeat):
        output = self(spec)
        output = output[onsets == 1]
        target = isbeat[onsets == 1]
        loss = self.loss_function(output, target)
        return loss

    def loss_from_dataset(self, dataset, batch_size=16):
        dataloader = DataLoader(dataset, batch_size=batch_size)
        loss = 0
        with torch.no_grad():
            loss = 0
            for specs, onsets, isbeat in dataloader:
                loss += self.loss(spec, onsets, isbeat).item()
            loss /= len(dataloader)
        return loss
    
    def learn(self, spec, onsets, isbeat):
        self.optimizer.zero_grad()
        output = self(spec)
        output = output[onsets == 1]
        target = isbeat[onsets == 1]
        loss = self.loss_function(output, target)
        loss.backward()
        self.optimizer.step()
        
        predic = torch.argmax(output, dim=1)
        accuracy = torch.sum(predic == target).item() / predic.shape.numel()
        
        return loss.item(), accuracy
    
    def fit(self, dataset, batch_size=1, epochs=1):
        loss_hist = np.zeros((epochs, -(-len(dataset) // batch_size)))
        accu_hist = np.zeros((epochs, -(-len(dataset) // batch_size)))
        for e in range(epochs):
            start = time.time()
            
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            for i, (spec, onsets, isbeat) in enumerate(dataloader):
                loss, accuracy = self.learn(spec, onsets, isbeat)
                loss_hist[e, i] = loss
                accu_hist[e, i] = accuracy
            
            end = time.time()
            t = end - start
            eta = str(datetime.timedelta(seconds=int(t * (epochs - e - 1))))
            print(f'| Epoch: {e + 1:{len(str(epochs))}} | ', end='')
            print(f'Loss: {np.mean(loss_hist[e]):7.4f} | ', end='')
            print(f'Accuracy: {np.mean(accu_hist[e]):5.4f} | ', end='')
            print(f'{t / len(dataloader):.2f} s/b | Eta: {eta} |')
        return loss_hist, accu_hist
    
    def predict(self, specs, onsets):
        """So far only works if batch_size = 1"""
        with torch.no_grad():
            output = model(specs)
            output = output[onsets == 1]
            pred_t = torch.argmax(output, dim=1)
            onsets_frames = np.argwhere(onsets.squeeze(0) == 1).squeeze(0)
            beats_frames = onsets_frames[pred_t == 1]
            pred = torch.zeros_like(onsets)
            pred[:, beats_frames] = 1
        return pred
    
    def evaluate(self, specs, onsets, isbeat):
        with torch.no_grad():
            output = model(specs)
            output = output[onsets == 1]
            target = isbeat[onsets == 1]
            predic = torch.argmax(output, dim=1)
            
            tn = torch.sum((predic == 0) & (target == 0)).item()
            fp = torch.sum((predic == 1) & (target == 0)).item()
            fn = torch.sum((predic == 0) & (target == 1)).item()
            tp = torch.sum((predic == 1) & (target == 1)).item()
        return tn, fp, fn, tp
    
    def evaluate_from_dataset(self, dataset, batch_size=16):
        dataloader = DataLoader(dataset, batch_size=batch_size)
        ttn = 0
        tfp = 0
        tfn = 0
        ttp = 0
        for specs, onsets, isbeat in dataloader:
            tn, fp, fn, tp = self.evaluate(specs, onsets, isbeat)
            ttn += tn
            tfp += fp
            tfn += fn
            ttp += tp
        return ttn, tfp, tfn, ttp
    
    def freeze(self):
        for p in self.parameters():
            p.requires_grad = False
            
    def unfreeze(self):
        for p in self.parameters():
            p.requires_grad = True

class ToTensor(object):
    
    def __call__(self, sample):
        spec_np, onsets_np, isbeat_np = sample
        
        # Normalize to [-1, 1]
        spec_np = 2 * (spec_np - spec_np.min()) / (spec_np.max() - spec_np.min()) - 1
        spec = torch.tensor(spec_np.T, dtype=torch.float, device=device)
        
        onsets = torch.zeros(spec.shape[0], dtype=torch.long, device=device)
        isbeat = torch.zeros(spec.shape[0], dtype=torch.long, device=device)
        
        onsets[onsets_np] = 1
        isbeat[onsets_np[isbeat_np == 1]] = 1
        
        return spec, onsets, isbeat

totensor = ToTensor()
    
def beat_track(isbeat):
    onset_envelope = isbeat.squeeze(0).numpy()
    tempo, bt = librosa.beat.beat_track(
                            sr=sr, 
                            onset_envelope=onset_envelope, 
                            hop_length=hl, 
                            tightness=800)
    return bt

# Visualization of the dataset

Take a subset of the GTZAN dataset preprocessed using `preprocess-GTZAN` and split it into a train set and a validation set.

In [ ]:
dataset = GTZAN(937, 'country', 20, getbeats=True)

Visualize an example.

In [ ]:
spec, onsets, isbeat, beats = dataset[np.random.randint(len(dataset))]

In [ ]:
showspec(spec)

In [ ]:
showdata(spec, onsets, isbeat, beats, duration=10)

# Find learning rate

In [ ]:
model = BeatTracker()
model.to(device)

In [ ]:
dataset = GTZAN(style='disco', idxs=20, transform=totensor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
n_mb = len(dataloader) # number of minibatches
lr = (10. ** -np.arange(0, 10, 10 / n_mb))[::-1]

In [ ]:
losses = np.zeros(n_mb)
for i, (specs, onsets, isbeat) in enumerate(dataloader):
    print(f'{i + 1}/{n_mb}')
    model.set_lr(lr[i])
    loss, accuracy = model.learn(specs, onsets, isbeat)
    losses[i] = loss

In [ ]:
plt.semilogx(lr, losses);

# Training of the model

In [10]:
model = BeatTracker()
model.to(device)
print_params(model)

 ----------------------------------------------------------
| lstm.weight_ih_l0            | [512, 229]   |    117,248 |
| lstm.weight_hh_l0            | [512, 128]   |     65,536 |
| lstm.bias_ih_l0              | [512]        |        512 |
| lstm.bias_hh_l0              | [512]        |        512 |
| lstm.weight_ih_l0_reverse    | [512, 229]   |    117,248 |
| lstm.weight_hh_l0_reverse    | [512, 128]   |     65,536 |
| lstm.bias_ih_l0_reverse      | [512]        |        512 |
| lstm.bias_hh_l0_reverse      | [512]        |        512 |
| lstm.weight_ih_l1            | [512, 256]   |    131,072 |
| lstm.weight_hh_l1            | [512, 128]   |     65,536 |
| lstm.bias_ih_l1              | [512]        |        512 |
| lstm.bias_hh_l1              | [512]        |        512 |
| lstm.weight_ih_l1_reverse    | [512, 256]   |    131,072 |
| lstm.weight_hh_l1_reverse    | [512, 128]   |     65,536 |
| lstm.bias_ih_l1_reverse      | [512]        |        512 |
| lstm.bias_hh_l1_reverse

In [11]:
dataset = GTZAN(style='disco', idxs=20, transform=totensor)

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
trainset, validset = random_split(dataset, [train_size, valid_size])

print(f'Train size: {train_size}')
print(f'Valid size: {valid_size}')

Train size: 16
Valid size: 4


In [12]:
confusion(*model.evaluate_from_dataset(validset))

 -------------------
| tp: 192 | fp: 245 |
| fn:  14 | tn:  24 |
 -------------------
Precision: 0.4394
   Recall: 0.9320
 Accuracy: 0.4547


In [57]:
confusion(*model.evaluate_from_dataset(trainset, 16))

 ---------------------
| tp:  840 | fp: 1106 |
| fn:   78 | tn:  105 |
 ---------------------
Precision: 0.4317
   Recall: 0.9150
 Accuracy: 0.4439


In [ ]:
lost_hist, accu_hist = model.fit(trainset, batch_size=3, epochs=1)

In [ ]:
plt.plot(np.mean(lost_hist, axis=1));

In [ ]:
confusion(*model.evaluate_from_dataset(validset))

In [ ]:
confusion(*model.evaluate_from_dataset(trainset))

In [ ]:
#torch.save(model.state_dict(), './data/model_02.pt')

# Test

In [ ]:
from GTZAN import *
from IPython.display import Audio

def unibatch(spec_np, onsets_np, isbeat_np):
    """ Get a minibatch of one sample from the original numpy data."""
    spec, onsets, isbeat = totensor((spec_np, onsets_np, isbeat_np))
    spec = spec.unsqueeze(0)
    onsets = onsets.unsqueeze(0)
    isbeat = isbeat.unsqueeze(0)
    return spec, onsets, isbeat

Load the pretrained model and freeze its parameters.

In [ ]:
model = BeatTracker()
model.load_state_dict(torch.load('./data/model_02.pt', map_location=device))
model.freeze()

Take a random example from the GTZAN dataset.

In [ ]:
s = np.random.choice(styles)
n = np.random.randint(20)
print(filename(s, n))

Compute the beat track from the RNN algorithm and compare with librosa and ground truth.

In [ ]:
# RNN
spec_np, onsets_np, isbeat_np, beats_np = loadGTZAN(s, n, True)
spec, onsets, isbeat = unibatch(spec_np, onsets_np, isbeat_np)
pred = model.predict(spec, onsets)
bt = beat_track(pred)
bt_times = librosa.frames_to_time(bt, sr, hl)

In [ ]:
# Librosa
onset_envelope = librosa.onset.onset_strength(S=spec_np)
tempo, bt_lib = librosa.beat.beat_track(onset_envelope=onset_envelope)
bt_lib_times = librosa.frames_to_time(bt_lib, sr, hl)

In [ ]:
plt.figure(figsize=(16, 2))
plt.vlines(bt_lib_times, 2, 3, color='b', label='Librosa')
plt.vlines(beats_np, 1, 2, color='g', label='Ground truth')
plt.vlines(bt_times, 0, 1, color='r', label='RNN')
plt.xlim(0, 30)
plt.ylim(0, 3)
plt.legend(frameon=True, framealpha=0.75, bbox_to_anchor=(1.15, 1));

Listen to the three different beat tracks.

In [ ]:
wav = librosa.load(filename_audio(s, n), sr=sr)[0]
Audio(wav, rate=sr)

In [ ]:
# RNN
click = librosa.clicks(bt_times, length=len(wav), sr=sr)
Audio(wav + click, rate=sr)

In [ ]:
# Librosa
click = librosa.clicks(bt_lib_times, length=len(wav), sr=sr)
Audio(wav + click, rate=sr)

In [ ]:
# Ground truth (sort of)
click = librosa.clicks(beats_np, length=len(wav), sr=sr)
Audio(wav + click, rate=sr)